In [ ]:
import json
import os
import numpy as np
import tensorflow as tf
from pathlib2 import Path
import preprocess
from data_loader import create_dataset, create_inputs

In [ ]:
def prepare_data(data_dir, language):
    if language == "Chinese":
        train_path = data_dir / "train_cn.json"
        test_path = data_dir / "test_cn.json"
        dev_path = data_dir / "dev_cn.json"

    elif language == "English":
        train_path = data_dir / "train_en.json"
        test_path = data_dir / "test_en.json"
        dev_path = data_dir / "dev_en.json"

    else:
        raise ValueError("Language must be English or Chinese")

    if not os.path.isfile(test_path):
        test_path = None

    return train_path, dev_path, test_path

data_dir = "./dataset"
language = "Chinese"
train_path, dev_path, test_path = prepare_data(data_dir=Path(data_dir), language=language)
train_path, dev_path, test_path

In [ ]:
model_input, customer_labels, helpdesk_labels, _, embedding_size  = create_inputs(json_path=train_path, plm="BERT",
                                                max_len=512,
                                                is_train=True, language="Chinese")

In [ ]:
inputs = (model_input, customer_labels, helpdesk_labels, [])
dataset, max_turn_number = create_dataset(inputs, "nugget", 10000)
a = list(dataset.take(1))

In [ ]:
input_ids = a[0][0]["input_ids"]
input_mask = a[0][0]["input_mask"]
input_type_ids = a[0][0]["input_type_ids"]
inputs = [input_ids, input_mask, input_type_ids]

In [ ]:
from model import Bert
plm = Bert(language=language, embedding_size=embedding_size)

In [ ]:
x = plm(inputs)

In [ ]:
input_type_ids.shape

In [ ]:
x.shape

In [ ]:
x

In [ ]:
sentence_ids = a[0][0]["sentence_ids"]
sentence_masks = a[0][0]["sentence_masks"]
sentence_ids.shape, sentence_masks.shape

In [ ]:
x.shape, sentence_ids.shape

In [ ]:
tf.gather(x, indices=sentence_ids, batch_dims=1) 

In [ ]:
sentence_ids[0]

In [ ]:
res = tf.gather(x, indices=sentence_ids, batch_dims=1)
print(res[0] == tf.gather(x[0], indices=sentence_ids[0]))

In [ ]:
res[0]

In [ ]:
tf.gather(x[0], indices=sentence_ids[0])

In [ ]:
res

In [ ]:
res2 = res * tf.cast(sentence_masks[:, :, None], dtype=tf.float32)

In [ ]:
res2

In [ ]:
from model import TransformerEncoder
encoder = TransformerEncoder(d_model=768, num_heads=8, d_ff=200, maximum_position_encoding=10000, rate=0.1, num_layers=2)

In [ ]:
posemb = encoder.pos_encoding[:, :7, :]
posemb.shape

In [ ]:
posemb

In [ ]:
res2[1]

In [ ]:
res2 += posemb

In [ ]:
res2[1]

In [ ]:
sentence_masks = 1 - sentence_masks

In [ ]:
sentence_masks

In [ ]:
sentence_masks = tf.expand_dims(sentence_masks, axis=1)

In [ ]:
sentence_masks

In [ ]:
def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)

    # add extra dimensions to add the padding
    # to the attention logits.
    return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

sentence_masks = create_padding_mask(sentence_masks)
sentence_masks

In [ ]:
for i in range(encoder.num_layers):
    res2 = encoder.enc_layers[i](res2, training=True, mask=sentence_masks)

In [ ]:
res2.shape

In [ ]:
customer_indices = tf.range(start=0, delta=2, limit=max_turn_number)
helpdesk_indices = tf.range(start=1, delta=2, limit=max_turn_number)

In [ ]:
res2

In [ ]:
cres = tf.gather(res2, indices=customer_indices, axis=1)

In [ ]:
hres = tf.gather(res2, indices=helpdesk_indices, axis=1)

In [ ]:
cres.shape, hres.shape

In [ ]:
customer_dense = tf.keras.layers.Dense(5, activation='gelu')

In [ ]:
clogits = customer_dense(cres) * tf.cast(cust_mask[:, :, None], tf.float32)

In [ ]:
clogits

In [ ]:
xxxx = tf.nn.softmax(clogits)

In [ ]:
tf.math.reduce_sum(xxxx, axis=-1)

In [ ]:
tf.nn.softmax(clogits)

In [ ]:
a[0][1]["customer_labels"]

In [ ]:
tf.math.reduce_sum(a[0][1]["customer_labels"], axis=-1)

In [ ]:
cus_loss = tf.nn.softmax_cross_entropy_with_logits(a[0][1]["customer_labels"], clogits, axis=-1)

In [ ]:
cus_loss.shape

In [ ]:
sentence_masks

In [ ]:
# customer_indices
cust_mask = tf.gather(sentence_masks, axis=1, indices=customer_indices)
cust_mask

In [ ]:
help_mask = tf.gather(sentence_masks, axis=1, indices=helpdesk_indices)
help_mask

In [ ]:
cus_loss.shape, cust_mask.shape

In [ ]:
cus_loss = cus_loss * tf.cast(cust_mask, tf.float32)

In [ ]:
cus_loss

In [ ]:
xx = tf.reduce_sum(cus_loss, axis=-1)
xx

In [ ]:
xxx = tf.reduce_mean(xx)

In [ ]:
xxx

In [ ]:
cusres = tf.nn.softmax(clogits, axis=-1)
cusres.shape, a[0][1]["customer_labels"].shape

In [ ]:
batch_sum = tf.math.reduce_sum(cusres, axis=-1)
batch_sum

In [ ]:
cusres2 = tf.keras.activations.softmax(clogits, axis=-1)
tf.math.reduce_sum(cusres2, axis=-1)